# Machine translation
    . The training and testing files are downloaded from https://nlp.stanford.edu/projects/nmt/
    

In [40]:
from __future__ import print_function
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import sys
from keras.preprocessing.text import text_to_word_sequence
from keras.models import Sequential
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam, RMSprop
from nltk import FreqDist
import os
import datetime

In [41]:
MAX_LEN = 100
VOCAB_SIZE = 5000
BATCH_SIZE = 300
LAYER_NUM = 4
HIDDEN_DIM = 500
NB_EPOCH = 20
MODE = 'train'

In [42]:

def load_data(source, dist, max_len, vocab_size):

    # Reading raw text from source and destination files
    f = open(source, 'r', encoding="utf8")
    X_data = f.read()
    f.close()
    f = open(dist, 'r', encoding="utf8")
    y_data = f.read()
    f.close()

    # Splitting raw text into array of sequences
    X = [text_to_word_sequence(x)[::-1] for x, y in zip(X_data.split('\n'), y_data.split('\n')) if len(x) > 0 and len(y) > 0 and len(x) <= max_len and len(y) <= max_len]
    y = [text_to_word_sequence(y) for x, y in zip(X_data.split('\n'), y_data.split('\n')) if len(x) > 0 and len(y) > 0 and len(x) <= max_len and len(y) <= max_len]

    # Creating the vocabulary set with the most common words
    dist = FreqDist(np.hstack(X))
    X_vocab = dist.most_common(vocab_size-1)
    dist = FreqDist(np.hstack(y))
    y_vocab = dist.most_common(vocab_size-1)

    # Creating an array of words from the vocabulary set, we will use this array as index-to-word dictionary
    X_ix_to_word = [word[0] for word in X_vocab]
    # Adding the word "ZERO" to the beginning of the array
    X_ix_to_word.insert(0, 'ZERO')
    # Adding the word 'UNK' to the end of the array (stands for UNKNOWN words)
    X_ix_to_word.append('UNK')

    # Creating the word-to-index dictionary from the array created above
    X_word_to_ix = {word:ix for ix, word in enumerate(X_ix_to_word)}

    # Converting each word to its index value
    for i, sentence in enumerate(X):
        for j, word in enumerate(sentence):
            if word in X_word_to_ix:
                X[i][j] = X_word_to_ix[word]
            else:
                X[i][j] = X_word_to_ix['UNK']

    y_ix_to_word = [word[0] for word in y_vocab]
    y_ix_to_word.insert(0, 'ZERO')
    y_ix_to_word.append('UNK')
    y_word_to_ix = {word:ix for ix, word in enumerate(y_ix_to_word)}
    for i, sentence in enumerate(y):
        for j, word in enumerate(sentence):
            if word in y_word_to_ix:
                y[i][j] = y_word_to_ix[word]
            else:
                y[i][j] = y_word_to_ix['UNK']
    return (X, len(X_vocab)+2, X_word_to_ix, X_ix_to_word, y, len(y_vocab)+2, y_word_to_ix, y_ix_to_word)


In [50]:
def load_test_data(source, X_word_to_ix, max_len):
    f = open(source, 'r', encoding="utf8")
    X_data = f.read()
    f.close()

    X = [text_to_word_sequence(x)[::-1] for x in X_data.split('\n') if len(x) > 0 and len(x) <= max_len]
    for i, sentence in enumerate(X):
        for j, word in enumerate(sentence):
            if word in X_word_to_ix:
                X[i][j] = X_word_to_ix[word]
            else:
                X[i][j] = X_word_to_ix['UNK']
    return X

def create_model(X_vocab_len, X_max_len, y_vocab_len, y_max_len, hidden_size, num_layers):
    model = Sequential()

    # Creating encoder network
    model.add(Embedding(X_vocab_len, 1000, input_length=X_max_len, mask_zero=True))
    model.add(LSTM(hidden_size))
    model.add(RepeatVector(y_max_len))

    # Creating decoder network
    for _ in range(num_layers):
        model.add(LSTM(hidden_size, return_sequences=True))
    model.add(TimeDistributed(Dense(y_vocab_len)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])
    return model

def process_data(word_sentences, max_len, word_to_ix):
    # Vectorizing each element in each sequence
    sequences = np.zeros((len(word_sentences), max_len, len(word_to_ix)))
    for i, sentence in enumerate(word_sentences):
        for j, word in enumerate(sentence):
            sequences[i, j, word] = 1.
    return sequences

def find_checkpoint_file(folder):
    checkpoint_file = [f for f in os.listdir(folder) if 'checkpoints.h5' in f]
    if len(checkpoint_file) == 0:
        return []
    modified_time = [os.path.getmtime(f) for f in checkpoint_file]
    return checkpoint_file[np.argmax(modified_time)]

In [51]:
 X, X_vocab_len, X_word_to_ix, X_ix_to_word, y, y_vocab_len, y_word_to_ix, y_ix_to_word = load_data('train.en', 'train.cs', MAX_LEN, VOCAB_SIZE)

In [52]:
# Finding the length of the longest sequence
X_max_len = max([len(sentence) for sentence in X])
y_max_len = max([len(sentence) for sentence in y])

# Padding zeros to make all sequences have a same length with the longest one
print('[INFO] Zero padding...')
X = pad_sequences(X, maxlen=X_max_len, dtype='int32')
y = pad_sequences(y, maxlen=y_max_len, dtype='int32')

# Creating the network model
print('[INFO] Compiling model...')
model = create_model(X_vocab_len, X_max_len, y_vocab_len, y_max_len, HIDDEN_DIM, LAYER_NUM)

[INFO] Zero padding...
[INFO] Compiling model...


In [54]:
# Finding trained weights of previous epoch if any
saved_weights = find_checkpoint_file('d:\\nmt\\')

# Training only if we chose training mode
if MODE == 'train':
    k_start = 1

    # If any trained weight was found, then load them into the model
    if len(saved_weights) != 0:
        print('[INFO] Saved weights found, loading...')
        epoch = saved_weights[saved_weights.rfind('_')+1:saved_weights.rfind('.')]
        model.load_weights(saved_weights)
        k_start = int(epoch) + 1

    i_end = 0
    for k in range(k_start, NB_EPOCH+1):
        # Shuffling the training data every epoch to avoid local minima
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X = X[indices]
        y = y[indices]

        # Training 1000 sequences at a time
        for i in range(0, len(X), 1000):
            if i + 1000 >= len(X):
                i_end = len(X)
            else:
                i_end = i + 1000
            y_sequences = process_data(y[i:i_end], y_max_len, y_word_to_ix)

            print('[INFO] Training model: epoch {}th {}/{} samples'.format(k, i, len(X)))
            model.fit(X[i:i_end], y_sequences, batch_size=BATCH_SIZE, nb_epoch=1, verbose=2)
        model.save_weights('checkpoint_epoch_{}.hdf5'.format(k))



[INFO] Training model: epoch 1th 0/49999 samples


d:\Anaconda3\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
11s - loss: 10.1082 - acc: 0.6535
[INFO] Training model: epoch 1th 1000/49999 samples
Epoch 1/1
5s - loss: 5.7646 - acc: 0.7493
[INFO] Training model: epoch 1th 2000/49999 samples
Epoch 1/1
5s - loss: 4.3664 - acc: 0.7480
[INFO] Training model: epoch 1th 3000/49999 samples
Epoch 1/1
5s - loss: 3.8787 - acc: 0.7468
[INFO] Training model: epoch 1th 4000/49999 samples
Epoch 1/1
5s - loss: 3.9490 - acc: 0.7455
[INFO] Training model: epoch 1th 5000/49999 samples
Epoch 1/1
5s - loss: 3.9972 - acc: 0.7470
[INFO] Training model: epoch 1th 6000/49999 samples
Epoch 1/1
5s - loss: 4.0554 - acc: 0.7475
[INFO] Training model: epoch 1th 7000/49999 samples
Epoch 1/1
5s - loss: 4.1061 - acc: 0.7475
[INFO] Training model: epoch 1th 8000/49999 samples
Epoch 1/1
5s - loss: 4.1717 - acc: 0.7475
[INFO] Training model: epoch 1th 9000/49999 samples
Epoch 1/1
5s - loss: 4.2155 - acc: 0.7485
[INFO] Training model: epoch 1th 10000/49999 samples
Epoch 1/1
6s - loss: 4.0972 - acc: 0.7490
[INFO] Training

15s - loss: 4.0391 - acc: 0.7490
[INFO] Training model: epoch 2th 37000/49999 samples
Epoch 1/1
15s - loss: 4.0731 - acc: 0.7455
[INFO] Training model: epoch 2th 38000/49999 samples
Epoch 1/1
18s - loss: 4.0636 - acc: 0.7470
[INFO] Training model: epoch 2th 39000/49999 samples
Epoch 1/1
19s - loss: 4.0730 - acc: 0.7468
[INFO] Training model: epoch 2th 40000/49999 samples
Epoch 1/1
19s - loss: 4.0446 - acc: 0.7488
[INFO] Training model: epoch 2th 41000/49999 samples
Epoch 1/1
19s - loss: 4.0509 - acc: 0.7468
[INFO] Training model: epoch 2th 42000/49999 samples
Epoch 1/1
15s - loss: 4.0696 - acc: 0.7478
[INFO] Training model: epoch 2th 43000/49999 samples
Epoch 1/1
15s - loss: 4.0507 - acc: 0.7490
[INFO] Training model: epoch 2th 44000/49999 samples
Epoch 1/1
15s - loss: 4.0554 - acc: 0.7460
[INFO] Training model: epoch 2th 45000/49999 samples
Epoch 1/1
18s - loss: 4.0343 - acc: 0.7483
[INFO] Training model: epoch 2th 46000/49999 samples
Epoch 1/1
19s - loss: 4.0468 - acc: 0.7483
[INFO] 

19s - loss: 4.0261 - acc: 0.7488
[INFO] Training model: epoch 4th 23000/49999 samples
Epoch 1/1
20s - loss: 4.0395 - acc: 0.7485
[INFO] Training model: epoch 4th 24000/49999 samples
Epoch 1/1
20s - loss: 4.0263 - acc: 0.7485
[INFO] Training model: epoch 4th 25000/49999 samples
Epoch 1/1
19s - loss: 4.0576 - acc: 0.7475
[INFO] Training model: epoch 4th 26000/49999 samples
Epoch 1/1
18s - loss: 4.0549 - acc: 0.7470
[INFO] Training model: epoch 4th 27000/49999 samples
Epoch 1/1
17s - loss: 4.0536 - acc: 0.7475
[INFO] Training model: epoch 4th 28000/49999 samples
Epoch 1/1
19s - loss: 4.0339 - acc: 0.7478
[INFO] Training model: epoch 4th 29000/49999 samples
Epoch 1/1
20s - loss: 4.0249 - acc: 0.7490
[INFO] Training model: epoch 4th 30000/49999 samples
Epoch 1/1
19s - loss: 4.0260 - acc: 0.7478
[INFO] Training model: epoch 4th 31000/49999 samples
Epoch 1/1
20s - loss: 4.0474 - acc: 0.7478
[INFO] Training model: epoch 4th 32000/49999 samples
Epoch 1/1
16s - loss: 4.0226 - acc: 0.7488
[INFO] 

...

16s - loss: 2.6395 - acc: 0.7480
[INFO] Training model: epoch 13th 1000/49999 samples
Epoch 1/1
18s - loss: 2.6482 - acc: 0.7485
[INFO] Training model: epoch 13th 2000/49999 samples
Epoch 1/1
21s - loss: 2.6269 - acc: 0.7488
[INFO] Training model: epoch 13th 3000/49999 samples
Epoch 1/1
21s - loss: 2.6463 - acc: 0.7485
[INFO] Training model: epoch 13th 4000/49999 samples
Epoch 1/1
20s - loss: 2.6490 - acc: 0.7475
[INFO] Training model: epoch 13th 5000/49999 samples
Epoch 1/1
21s - loss: 2.6588 - acc: 0.7475
[INFO] Training model: epoch 13th 6000/49999 samples
Epoch 1/1
21s - loss: 2.6487 - acc: 0.7475
[INFO] Training model: epoch 13th 7000/49999 samples
Epoch 1/1
20s - loss: 2.6449 - acc: 0.7480
[INFO] Training model: epoch 13th 8000/49999 samples
Epoch 1/1
18s - loss: 2.6356 - acc: 0.7480
[INFO] Training model: epoch 13th 9000/49999 samples
Epoch 1/1
17s - loss: 2.6442 - acc: 0.7493
[INFO] Training model: epoch 13th 10000/49999 samples
Epoch 1/1
20s - loss: 2.6362 - acc: 0.7490
[INFO]

20s - loss: 2.6541 - acc: 0.7480
[INFO] Training model: epoch 14th 36000/49999 samples
Epoch 1/1
19s - loss: 2.6496 - acc: 0.7478
[INFO] Training model: epoch 14th 37000/49999 samples
Epoch 1/1
20s - loss: 2.6630 - acc: 0.7480
[INFO] Training model: epoch 14th 38000/49999 samples
Epoch 1/1
21s - loss: 2.6436 - acc: 0.7488
[INFO] Training model: epoch 14th 39000/49999 samples
Epoch 1/1
19s - loss: 2.6575 - acc: 0.7483
[INFO] Training model: epoch 14th 40000/49999 samples
Epoch 1/1
17s - loss: 2.6413 - acc: 0.7483
[INFO] Training model: epoch 14th 41000/49999 samples
Epoch 1/1
21s - loss: 2.6557 - acc: 0.7480
[INFO] Training model: epoch 14th 42000/49999 samples
Epoch 1/1
20s - loss: 2.6730 - acc: 0.7480
[INFO] Training model: epoch 14th 43000/49999 samples
Epoch 1/1
20s - loss: 2.6412 - acc: 0.7488
[INFO] Training model: epoch 14th 44000/49999 samples
Epoch 1/1
20s - loss: 2.6598 - acc: 0.7470
[INFO] Training model: epoch 14th 45000/49999 samples
Epoch 1/1
18s - loss: 2.6492 - acc: 0.74

19s - loss: 2.5861 - acc: 0.7490
[INFO] Training model: epoch 16th 21000/49999 samples
Epoch 1/1
16s - loss: 2.5878 - acc: 0.7488
[INFO] Training model: epoch 16th 22000/49999 samples
Epoch 1/1
19s - loss: 2.5877 - acc: 0.7485
[INFO] Training model: epoch 16th 23000/49999 samples
Epoch 1/1
20s - loss: 2.5955 - acc: 0.7488
[INFO] Training model: epoch 16th 24000/49999 samples
Epoch 1/1
19s - loss: 2.5974 - acc: 0.7478
[INFO] Training model: epoch 16th 25000/49999 samples
Epoch 1/1
17s - loss: 2.5899 - acc: 0.7480
[INFO] Training model: epoch 16th 26000/49999 samples
Epoch 1/1
17s - loss: 2.6112 - acc: 0.7463
[INFO] Training model: epoch 16th 27000/49999 samples
Epoch 1/1
20s - loss: 2.5966 - acc: 0.7488
[INFO] Training model: epoch 16th 28000/49999 samples
Epoch 1/1
20s - loss: 2.6189 - acc: 0.7458
[INFO] Training model: epoch 16th 29000/49999 samples
Epoch 1/1
20s - loss: 2.6043 - acc: 0.7483
[INFO] Training model: epoch 16th 30000/49999 samples
Epoch 1/1
21s - loss: 2.5897 - acc: 0.74

17s - loss: 2.4986 - acc: 0.7483
[INFO] Training model: epoch 18th 6000/49999 samples
Epoch 1/1
17s - loss: 2.4968 - acc: 0.7485
[INFO] Training model: epoch 18th 7000/49999 samples
Epoch 1/1
20s - loss: 2.5283 - acc: 0.7463
[INFO] Training model: epoch 18th 8000/49999 samples
Epoch 1/1
20s - loss: 2.5095 - acc: 0.7500
[INFO] Training model: epoch 18th 9000/49999 samples
Epoch 1/1
20s - loss: 2.5567 - acc: 0.7478
[INFO] Training model: epoch 18th 10000/49999 samples
Epoch 1/1
18s - loss: 2.5368 - acc: 0.7478
[INFO] Training model: epoch 18th 11000/49999 samples
Epoch 1/1
17s - loss: 2.5338 - acc: 0.7490
[INFO] Training model: epoch 18th 12000/49999 samples
Epoch 1/1
18s - loss: 2.5589 - acc: 0.7460
[INFO] Training model: epoch 18th 13000/49999 samples
Epoch 1/1
20s - loss: 2.5223 - acc: 0.7488
[INFO] Training model: epoch 18th 14000/49999 samples
Epoch 1/1
19s - loss: 2.5357 - acc: 0.7485
[INFO] Training model: epoch 18th 15000/49999 samples
Epoch 1/1
19s - loss: 2.5597 - acc: 0.7455
[

17s - loss: 2.5487 - acc: 0.7485
[INFO] Training model: epoch 19th 41000/49999 samples
Epoch 1/1
20s - loss: 2.5751 - acc: 0.7473
[INFO] Training model: epoch 19th 42000/49999 samples
Epoch 1/1
21s - loss: 2.5759 - acc: 0.7460
[INFO] Training model: epoch 19th 43000/49999 samples
Epoch 1/1
20s - loss: 2.5605 - acc: 0.7488
[INFO] Training model: epoch 19th 44000/49999 samples
Epoch 1/1
20s - loss: 2.5654 - acc: 0.7483
[INFO] Training model: epoch 19th 45000/49999 samples
Epoch 1/1
17s - loss: 2.5592 - acc: 0.7480
[INFO] Training model: epoch 19th 46000/49999 samples
Epoch 1/1
18s - loss: 2.5872 - acc: 0.7468
[INFO] Training model: epoch 19th 47000/49999 samples
Epoch 1/1
21s - loss: 2.5665 - acc: 0.7488
[INFO] Training model: epoch 19th 48000/49999 samples
Epoch 1/1
20s - loss: 2.5763 - acc: 0.7473
[INFO] Training model: epoch 19th 49000/49999 samples
Epoch 1/1
20s - loss: 2.5595 - acc: 0.7492
[INFO] Training model: epoch 20th 0/49999 samples
Epoch 1/1
18s - loss: 2.4361 - acc: 0.7488
[

In [55]:
#save the model and weights in Yaml format
from keras.models import model_from_json
from keras.models import model_from_yaml
import numpy as np
import os

np.random.seed(12345)

model.save('d:\\nmt\\nmt_model.h5')
model.save_weights('d:\\nmt\\weights_nmt_yaml.h5')
# serialize model to YAML
model_yaml = model.to_yaml()
with open("d:\\nmt\\nmt_model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("d:\\nmt\\weights_nmt_yaml.h5")
print("Saved model to disk")

# load YAML and create model
yaml_file = open('d:\\nmt\\nmt_model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("d:\\nmt\\weights_nmt_yaml.h5")
print("Loaded model from disk")

Saved model to disk
Loaded model from disk


In [95]:
#load test data 
X_test = load_test_data('test.txt', X_word_to_ix, MAX_LEN)


In [99]:
X_test = pad_sequences(X_test, maxlen=X_max_len, dtype='int32')
model.load_weights('checkpoint_epoch_20.hdf5')

In [108]:
predictions = np.argmax(model.predict(X_test), axis=2)
sequences = []
for prediction in predictions:
    sequence = ' '.join([y_ix_to_word[index] for index in prediction if index > 0])
    sequences.append(sequence)
np.savetxt('nmt_submission.txt', sequences, fmt='%s')